In [1]:
import numpy as np
import sympy as sp
import pickle

from concurrent.futures import ProcessPoolExecutor, as_completed
from scipy.integrate import trapz,cumulative_trapezoid,quad

import os, time
from datetime import datetime

from qutip import *
import matplotlib.pyplot as plt
from sympy.physics.quantum import TensorProduct as ts
from sympy.physics.quantum.dagger import Dagger
from sympy.physics.matrices import msigma

from sympy import lambdify

In [2]:
def HCD(TT):
    sI,sx,sy,sz = sp.eye(2),msigma(1),msigma(2),msigma(3)
    
    t = sp.symbols(r't', real=True, positive=True)
    f = sp.Function('f')(t)
    g = sp.Function('g')(t)

    f, g = t/TT, t/TT

    sHi = (ts(sx,sx,sI)+ts(sy,sy,sI))
    sHm = (ts(sI,sx,sx)+ts(sI,sy,sy))
    sHf = (ts(sz,sI,sz) + ts(sI,sz,sz))

    sHad = (1-f)*sHi + f*(1-f)*sHm + g*sHf
    
    m = sHad.eigenvects(chop=True)
    
    e = [m[i//2][0] for i in range(len(m)*2)]
    ev_ = [[tup[2][0], tup[2][1]] for tup in m]
    ev = [val for sublist in ev_ for val in sublist]
    evN = [vec/sp.sqrt((Dagger(vec)*vec)[0]) for vec in ev] 

    sHcd = sum([1j*sp.diff(vec,t)*Dagger(vec) for vec in evN],sp.zeros(8,8))
    
    return sHcd

def calc_HCD_COMP(TT,ij):

    t0 = datetime.now()

    t = sp.symbols(r't', real=True, positive=True)

    H = HCD(TT)
            

    x = H[ij[0],ij[1]].evalf()

    lam_x = sp.lambdify(t, x, modules=['mpmath'])


    x_vals = np.linspace(0.0001, .99*TT, 10000, dtype=complex)

    y_vals = []
    for _x in x_vals:
        y_vals.append(complex(lam_x(_x)))
    
    
    path = r'data/HCD_components/' +str(TT) + '/'
    if not os.path.exists(path):
        os.makedirs(path)

    with open(path + str(ij[0]) + str(ij[1]), 'wb') as fp:
        pickle.dump([y_vals, TT, ij], fp) 

    tf = datetime.now()
    
    print(TT, ij[0], ij[1] , 'start = ',t0.strftime("%d %H:%M:%S"), 'stop ='  ,tf.strftime("%d %H:%M:%S"))

    return [y_vals, TT, ij]

In [8]:
H = HCD(1)
nonZero_index_Hcd = (np.transpose(np.nonzero(H))).tolist()
for i in nonZero_index_Hcd:
    if i[0] == i[1]:
        nonZero_index_Hcd.remove(i)

In [7]:
tau_list = [10**(-3+8*i*.005) for i in range(25,175)] #[0.01,...,9120] evolution time
tau_list[50]

1.0

In [21]:
arg_list = [(t,i) for t in tau_list for i in nonZero_index_Hcd]

In [ ]:
# Pegando o numero máximo de núcleos
max_cpu = os.cpu_count()
with ProcessPoolExecutor(max_workers=max_cpu) as executor:

    # Executando as funções e salvando as saídas
    futures = [executor.submit(calc_HCD_COMP, arg[0],arg[1]) for arg in arg_list]

      # Trecho útil pra ver quantas funções já foram executadas, mas pode apagar.
    r = 0
    for _ in as_completed(futures):
        r+=1
        if r%max_cpu==0:
            print(f'About {len(executor._pending_work_items)} tasks remain')
    print("")
            